In [0]:
!pip install rake-nltk
from collections import namedtuple
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.metrics.pairwise import polynomial_kernel
import pickle
import os

Datum = namedtuple('Datum', 'ref cite offsets author is_test facet year')
Offsets = namedtuple('Offsets', 'marker cite ref')
Article = namedtuple('Article', 'id xml sentences sections')


import nltk
nltk.download('stopwords')


from rake_nltk import Rake
from nltk.corpus import stopwords
import re

stop = set(stopwords.words('english'))

rakey = Rake(max_length=3)

def encode(sentence):
  return sentence

# latest tf-idf


global tf_vectorizer


def build_model(cite_paper, ref_paper, n=1):
    tf = TfidfVectorizer(analyzer='word', ngram_range=(1, n), min_df=0, stop_words='english')
    sentences = list(cite_paper.sentences.values())
    sentences.extend(list(ref_paper.sentences.values()))
    tf_vectorizer = tf.fit(sentences)
    pickle.dump(tf_vectorizer, open("/content/drive/My Drive/qa_stuff/tf-idf_cache/tf_cite_{}_ref_{}_n_{}.pickle".format(cite_paper.id, ref_paper.id, n), 'wb'))
    return tf_vectorizer

def encode(sentence):
  return sentence

def get_similarity_score(cite_paper, ref_paper, sentence1, sentence2, n=1, kernel="none"):
    if os.path.exists("/content/drive/My Drive/qa_stuff/tf-idf_cache/tf_cite_{}_ref_{}_n_{}.pickle".format(cite_paper.id, ref_paper.id, n)):
        tf_vectorizer = pickle.load(open("/content/drive/My Drive/qa_stuff/tf-idf_cache/tf_cite_{}_ref_{}_n_{}.pickle".format(cite_paper.id, ref_paper.id, n), 'rb'))
    else:
        tf_vectorizer = build_model(cite_paper, ref_paper, n)

    tfidf_1 = tf_vectorizer.transform([sentence1])
    tfidf_2 = tf_vectorizer.transform([sentence2])
    if kernel == "none":
        return cosine_similarity(tfidf_1, tfidf_2).item()
    elif kernel == "linear":
        return linear_kernel(tfidf_1, tfidf_2).item()
    elif kernel == "poly_2":
        return polynomial_kernel(tfidf_1, tfidf_2, 2).item()
    elif kernel == "poly_3":
        return polynomial_kernel(tfidf_1, tfidf_2, 3).item()
    elif kernel == "rbf":
        return rbf_kernel(tfidf_1, tfidf_2).item()
# def get_similarity_score(sentence1, sentence2):
#     # doesn't do anything about frequency of words in a sentence
#     tokens1 = set(re.findall(r'[\w]+', sentence1.lower()))
#     tokens2 = set(re.findall(r'[\w]+', sentence2.lower()))
#     # print(sentence1)
#     rakey.extract_keywords_from_sentences(sentence1.replace("-", " ").lower().split())

#     keys1 = rakey.get_ranked_phrases()
#     #keys1 = [x.split() for x in keys1]
#     # print(keys1)
#     #keys1 = [item for sublist in keys1 for item in sublist]
#     # print(keys1)

#     rakey.extract_keywords_from_sentences(sentence2.replace("-", " ").lower().split())
#     keys2 = rakey.get_ranked_phrases()
#     #keys2 = [x.split() for x in keys2]
#     #keys2 = [item for sublist in keys2 for item in sublist]

#     #keys = set(keys1) and set(keys2)
#     tokens1 = set(keys1) - stop #- set(ref_keys)
#     tokens2 = set(keys2) - stop #- set(cite_keys)
#     #print(tokens1)
#     #print(tokens2)
#     #wefwefwef
#     # tokens1 = tokens1 - stop
#     # tokens2 = tokens2 - stop
  
#     return len(tokens1.intersection(tokens2)) / len(tokens1.union(tokens2))


from tqdm import tqdm_notebook as tqdm
accuracy = 0
total = 0
empty_citations = 0
empty_references = 0
import pickle

with open("/content/drive/My Drive/qa_stuff/processed-data-2019.json", 'rb') as f:
  dataset = pickle.load(f)

with open("/content/drive/My Drive/qa_stuff/processed-data-2018.json", 'rb') as f:
  dataset2 = pickle.load(f)

dataset =  dataset2 + dataset

articles = [x[1] for x in dataset]
n = 5



pbar = tqdm(dataset)
tp = 0
fp = 0
fn = 0
avg_score = 0
all_scores = []
for data in pbar:
    ref_article = data.ref
    citing_article = data.cite
    offsets = data.offsets
    citing_sentence_ids = offsets.cite
    true_ref_sentences = offsets.ref
    true_ref_sentence_ids = offsets.ref
    if tp > 0:
      p = tp/(max(tp+fp,1))
      r = tp/(max(tp+fn,1))
      f1 = 2*p*r/(p+r)
      pbar.set_description("Processing %.3f %.3f %.3f" %(p, r, f1))
    if citing_article.sentences:
      # raku.extract_keywords_from_sentences([citing_article.sentences[x] for x in citing_article.sentences])
      # cite_keys = raku.get_ranked_phrases()[:0]
      # raku.extract_keywords_from_sentences([ref_article.sentences[x] for x in ref_article.sentences])
      # ref_keys = raku.get_ranked_phrases()[:0]

      new_ids = [c for c in citing_sentence_ids]
      for c in citing_sentence_ids:
            # If additional context is reqd
            to_add  = 0
            extra = range(max(1, c-to_add), c)
            new_ids.extend(extra)
            extra = range(c+1, min(len(citing_article.sentences), c + to_add+1))
            new_ids.extend(extra)
      citing_sentence_ids = new_ids
      complete_citing_sentence = " ".join([citing_article.sentences[c] for c in citing_sentence_ids])
      ref_vs = [(x[0],encode(x[1])) for x in ref_article.sentences.items() if abs(len(x[1].split()) - len(complete_citing_sentence.split())) <=40]
      similarity_score = {}
      for i in range(1):#c in citing_sentence_ids:
        complete_citing_sentence = ' '.join([citing_article.sentences[x] for x in citing_sentence_ids])
        complete_citing_sentence = citing_article.sentences[c]
        #complete_citing_sentence = re.sub('\((.+?)\)', '', complete_citing_sentence)
        complete_citing_sentence = encode(complete_citing_sentence)
        for ref_id, ref_sentence in ref_vs: #(ref_article.sentences.items()):
            try:
                similarity_score[ref_id] = max(similarity_score.get(ref_id, 0) , get_similarity_score(citing_article, ref_article, ref_sentence, complete_citing_sentence, 1, "poly_2"))
                # similarity_score[ref_id] = max(similarity_score.get(ref_id, 0) , get_similarity_score(ref_sentence, complete_citing_sentence))
            except Exception as e:
                print(e)
                pass
      if similarity_score:
          sorted_similarity_score = sorted(similarity_score.items(), key=lambda item: -item[1])
          top_n = [s for s in sorted_similarity_score]
          top_n = {x[0]:x[1] for x in top_n[:3]}
          # if len(top_n) > 1:
          #   top_n = rerank2(top_n, ref_article, ' '.join([citing_article.sentences[x] for x in citing_sentence_ids]))

          fp += len(top_n)
          for x in true_ref_sentence_ids:
            if x in top_n:
              avg_score = (tp*avg_score + top_n[x])/(max(tp,1))
              all_scores.append(top_n[x])
              fp -= 1
              tp += 1
            else:
              #print(ref_article.sentences[x])
              #print([citing_article.sentences[x] for x in citing_sentence_ids])
              fn += 1
print(tp, fp, fn)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:116: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


In [0]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Wed Apr  1 03:48:36 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8     7W /  75W |      0MiB /  7611MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
import torch
cuda = torch.cuda.is_available()
device = torch.device("cuda" if cuda else "cpu")

# New Section

In [0]:
!pip install -U sentence-transformers
!pip install rake-nltk

     |████████████████████████████████| 61kB 9.6MB/s 
     |████████████████████████████████| 450kB 26.8MB/s 
     |████████████████████████████████| 1.0MB 67.3MB/s 
     |████████████████████████████████| 870kB 58.0MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.2.5.1-cp36-none-any.whl size=67076 sha256=01e8aa36878b49f7e2a6698ee53228d939ac96d30d2bd30c32430fe7753d29d7
  Stored in directory: /root/.cache/pip/wheels/22/ca/b4/7ca542b411730a8840f8e090df2ddacffa1c4dd9f209684c19
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=3b66df8f1091bae3624107bf49f0b06efdd3e306e1c1d74994afefd292b55469
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sentence-transformers sacremoses
  Created wheel for rake-nltk: filename=rake_nltk-1.0.4-py2.py3-none-any.whl size=7819 sha256=a7a91ae885b186d75dad374127440de25015b38e5e267fc1403e36865e09661d
  Stored 

In [0]:
from collections import namedtuple

Datum = namedtuple('Datum', 'ref cite offsets author is_test facet year')
Offsets = namedtuple('Offsets', 'marker cite ref')
Article = namedtuple('Article', 'id xml sentences sections')






In [0]:

import os
import copy
import xml.etree.cElementTree as ET
import logging
import codecs
from tqdm import tqdm
import re
import requests

DATA_ROOT = '/content/drive/My Drive/data'
sep = os.path.sep
ET.XMLParser(encoding="utf-8")

paper_load_fail = 0
annotation_load_fail = 0

l = logging.getLogger('load_parse')
replace_count = 0

# Gets names of folders where data is present. ex : "./data/Training-Set-2016/C90-2039_TRAIN"
def get_folders(data_root):
    folders = []
    global folder_skips
    for year_folder in filter(lambda x: os.path.isdir(data_root + sep + x), os.listdir(data_root)):
        year_folder = data_root + sep + year_folder
        for data_folder in os.listdir(year_folder):
            data_folder = year_folder + sep + data_folder
            if not os.path.isdir(data_folder) or '2018' not in data_folder or 'Test-Set-2018' in data_folder:
                continue
                # readme
            folders.append(data_folder)
    return folders


def load_article(filename):
    global paper_load_fail
    if not os.path.splitext(filename)[1] == '.xml':
        l.info("Skipping Non xml : " + filename)
        return
    l.info("parsing :" + filename)
    try:
        # Ignoring non UTF characters for now
        with codecs.open(filename, mode='r', encoding='utf-8', errors='ignore') as target_file:

            xml = ET.parse(target_file)
            parent_map = {c: p for p in xml.getroot().iter() for c in p}
            sentence_elements = list(xml.getroot().iter('S'))
            sentence_elements = [(x.text,
                                  parent_map[x].attrib['title'] if len(parent_map[x].attrib) > 0 else parent_map[x].tag,
                                  int(x.attrib['sid']) if 'sid' in x.attrib else 99)
                                 for x in
                                 sentence_elements]
            # TODO: Check if this is too memory inefficient. Should mostly be okay
            sentence_map = {x[2]: x[0] for i, x in enumerate(sentence_elements)}
            section_map = {x[2]: x[1] for i, x in enumerate(sentence_elements)}
            article = Article("", xml, sentence_map, section_map)
            return article
    except Exception as e:
        l.error("Error with : " + filename + " with ex : " + str(e))
        paper_load_fail += 1
        return Article("", ET.fromstring("<xml></xml>"), {}, {})


def newest_file(path):
    files = os.listdir(path)
    paths = [os.path.join(path, basename) for basename in files]
    return max(paths, key=os.path.getctime)


author_memo = {}
def load_folder_data(annotation_file, articles, is_test=False):
    global annotation_load_fail
    global author_memo
    data = []
    # try:
    #     year_matches = re.findall(".*Set[-]([0-9]{4}).*", annotation_file)
    #     year = int(year_matches[0]) if len(year_matches) > 0 else 2016 # Dev set
    # except:
    #     print("kk")
    with open(annotation_file, 'r') as f:
        for line in f:
            if len(line.strip()) == 0:
                continue
            try:
                parts = line.split(" | ")
                parts = [part.strip() for part in parts]
                ref_article_name = parts[1].split(":")[1].strip().upper().replace(".XML", "").replace(".TXT", "")
                ref_article = articles[ref_article_name]
                cite_article = articles["-".join(
                    parts[2].split(":")[1].strip().upper().replace("_", "-").replace(".XML", "").replace(".TXT",
                                                                                                         "").split("-")[
                    :2])]
                marker_offset = [int(x) for x in (
                    parts[5].split(":")[1].replace("'", "").replace("[", "").replace("]", "").strip()).replace(",",
                                                                                                               " ").split()]
                citation_offsets = [int(x) for x in (
                    parts[5].split(":")[1].replace("'", "").replace("[", "").replace("]", "").strip()).replace(",",
                                                                                                               " ").split()]
                ref_offsets = [int(x) for x in (
                    parts[7].split(":")[1].replace("'", "").replace("[", "").replace("]", "").strip()).replace(",",
                                                                                                               " ").split()]
                facet = parts[9].split(":")[1].strip()
                facet = [facet] if '[' not in facet else eval(facet)
                if ref_article_name not in author_memo:
                    url = "https://www.aclweb.org/anthology/{}.bib".format(ref_article_name)
                    info = requests.get(url=url).text
                    author_info = info.split('{')[1].split(',')[0].split('-')
                    author = get_formatted_author_info(author_info)
                    author_memo[ref_article_name] = author_info
                else:
                    author_info = author_memo[ref_article_name]
                    author = get_formatted_author_info(author_info)
                year = author_info[-2]
                # author = '' if len(parts) < 10 or ":" not in parts[10] else parts[10].split(":")[1].replace("|",
                #                                                                                             "").strip()
                reference, cite = get_clean_cite_and_ref(ref_article, cite_article, ref_offsets, citation_offsets,
                                                         author, year)
                # reference, cite = ref_article, cite_article
                d = Datum(reference, cite, Offsets(marker_offset, citation_offsets, ref_offsets), author,
                          is_test, facet, year)
                data.append(d)
            except Exception as e:
                l.error(e)
                annotation_load_fail += 1
    return data


def load_folder(folder_root):
    annotation_folder = folder_root + sep + "annotation"
    citance_dir = folder_root + sep + "Citance_XML"
    ref_name = folder_root.split(sep)[-1].replace("_TRAIN", "")
    ref_dir = folder_root + sep + "Reference_XML" + sep + ref_name + ".xml"
    articles = {
        '-'.join(os.path.splitext(x)[0].upper().replace("_", "-").split("-")[:2]): load_article(citance_dir + sep + x)
        for x in os.listdir(citance_dir)}

    assert len(articles) == len(os.listdir(citance_dir))

    ref_article = load_article(ref_dir)
    articles[ref_name.upper()] = ref_article
    # Hack
    for key in articles:
        if articles[key]:
            articles[key] = articles[key]._replace(id=key)

    annotation_file = newest_file(annotation_folder)
    folder_data = load_folder_data(annotation_file, articles)
    return folder_data


def load_all(root):
    print("Loading all folders, skipping 2019 and test set 2018 for now")
    folders = get_folders(root)
    print("Going to load :", len(folders))
    dataset = [load_folder(folder) for folder in tqdm(folders)]
    dataset = [data for datalist in dataset for data in datalist]
    print()
    print("Paper load fails due to xml errors : ", paper_load_fail)
    print("Annotation load fails due to random reasons", annotation_load_fail)
    print("Overall loaded ", len(dataset), " datapoints")
    return dataset


def get_clean_cite_and_ref(ref_article, cite_article, ref_offsets, citation_offsets, author, year):
    global replace_count
    ref = copy.copy(ref_article)
    cite = copy.copy(cite_article)

    # remove cites in ref
    for key, sentence in ref.sentences.items():
        ref.sentences[key] = get_cites(sentence)

    cite_sentence = " ".join([cite.sentences[c] for c in citation_offsets])
    cite_sentence = re.sub(r"\D(\d{4})\D", '', cite_sentence)  # regex for removing years
    cite_sentence = re.sub(r"\[[0-9]{1,3}\]", '', cite_sentence)  # regex for removing citation numbers
    translation = {ord(')'): None, ord('('): None, ord('.'): None,  ord(','): None, ord('!'): 'l'}
    cite_sentence = cite_sentence.translate(translation)
    author_info = author.split(" ")
    author_2 = None
    if author_info[-1] in ["et.al.", "etal", "Etal"]:
        author_1 = " ".join(author_info[:-1]) + " et al"
        if len(author_info) >= 2:
            author_2 = author_info[0] + " and " + author_info[1]
    elif len(author_info) >= 2 and author_info[-2] not in ["And", "and"]:
        author_1 = author_info[0] + " & " + author_info[1]
        author_2 = author_info[0] + " and " + author_info[1]
    else:
        author_1 = author_info[-1]

    citing_paper_text = author_1
    old_cite_sentence = cite_sentence
    cite_sentence = cite_sentence.replace(citing_paper_text, "##CITATION##")
    if cite_sentence != old_cite_sentence:
        replace_count += 1
    elif author_2:
        citing_paper_text = author_2
        cite_sentence = cite_sentence.replace(citing_paper_text, "##CITATION##")
        if cite_sentence != old_cite_sentence:
            replace_count += 1
        elif len(author_info) >= 2 and author_info[-2] not in ["And", "and"]:
            author_info.sort()
            author_1 = author_info[0] + " & " + author_info[1]
            author_2 = author_info[0] + " and " + author_info[1]
            citing_paper_text = author_1
            old_cite_sentence = cite_sentence
            cite_sentence = cite_sentence.replace(citing_paper_text, "##CITATION##")
            if cite_sentence != old_cite_sentence:
                replace_count += 1
            elif author_2:
                citing_paper_text = author_2
                cite_sentence = cite_sentence.replace(citing_paper_text, "##CITATION##")
                if cite_sentence != old_cite_sentence:
                    replace_count += 1
                else:
                    print("cite_sentence: {}\n old_cite_sentence: {}\n author: {}".format(cite_sentence,
                                                                                          old_cite_sentence, author))
            else:
                print("cite_sentence: {}\n old_cite_sentence: {}\n author: {}".format(cite_sentence,
                                                                                      old_cite_sentence, author))
        else:
            print("cite_sentence: {}\n old_cite_sentence: {}\n author: {}".format(cite_sentence, old_cite_sentence, author))
    else:
        print("cite_sentence: {}\n old_cite_sentence: {}\n author: {}".format(cite_sentence, old_cite_sentence, author))

    # remove all other cites in cite
    cite.sentences[citation_offsets[0]] = cite_sentence
    for key, sentence in cite.sentences.items():
        cite.sentences[key] = get_cites(sentence)

    if len(citation_offsets) > 1:
        for offset in citation_offsets[1:]:
            cite.sentences[offset] = ""
    print("replace_count: ", replace_count)
    return ref, cite


def get_formatted_author_info(author_info):
    author_details = " ".join(author_info)
    author_info = re.sub(r"[0-9]{4}", '_', author_details).split('_')[0].split()
    author = " ".join([x.capitalize() if x is not "and" else x for x in author_info])
    return author

def get_cites(sentence):
    regex = r"\(\D*\d{4}(;\D*\d{4})*\)"
    sentence = re.sub(regex, " ", sentence)
    return sentence

if __name__ == "__main__":
    logging.basicConfig(filename='example.log', level=logging.ERROR, filemode='w')
    print("Logging to example.log")
dataset = load_all(DATA_ROOT)


Logging to example.log
Loading all folders, skipping 2019 and test set 2018 for now


FileNotFoundError: ignored

In [0]:
dataset[0]

In [0]:
pip install transformers


     |████████████████████████████████| 501kB 38.9MB/s 
     |████████████████████████████████| 3.7MB 51.5MB/s 
     |████████████████████████████████| 870kB 61.7MB/s 
     |████████████████████████████████| 1.0MB 61.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=6650412c6ebd99525730053f180e28016e24dc1b9d6b7e8611eccee057e55b3d
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [0]:
from transformers import *

In [0]:
import torch 
import json
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader



class CiteModel(nn.Module):
    def __init__(self):
        super(CiteModel, self).__init__()
        self.tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
        self.model = RobertaModel.from_pretrained('roberta-base')
        self.fc1 = nn.Linear(in_features=768*4, out_features=1024)
        # self.bn1 = nn.BatchNorm1d(1024)
        self.dropout1 = nn.Dropout(0.1)
        self.act1 = nn.ReLU()
        self.fc2 = nn.Linear(in_features=1024, out_features=512)
        # self.bn2 = nn.BatchNorm1d(512)
        self.dropout2 = nn.Dropout(0.1)
        self.act2 = nn.ReLU()
        self.fc3 = nn.Linear(in_features=512, out_features=2)

    def forward(self, ref_sentences, cite_sentences):
        ref_encoded = self.tokenizer.batch_encode_plus([x.lower() for x in ref_sentences], add_special_tokens=True)['input_ids']
        cite_encoded = self.tokenizer.batch_encode_plus([x.lower() for x in cite_sentences], add_special_tokens=True)['input_ids']
        # print(ref_encoded)
        # print(cite_encoded)
        # sdfc
        ref_encoded = pad_to_max(ref_encoded)
        cite_encoded = pad_to_max(cite_encoded)
        # print(ref_encoded)
        # print(cite_encoded)
        # sdfc
        ref_encoded, cite_encoded = torch.LongTensor(ref_encoded), torch.LongTensor(cite_encoded)
        x, ref_encoded = self.model(ref_encoded.to(device))
        y, cite_encoded = self.model(cite_encoded.to(device))
        x = torch.cat([ref_encoded, cite_encoded, ref_encoded-cite_encoded, ref_encoded*cite_encoded], dim=1).to(device)
        x = self.act1(self.dropout1(self.fc1(x)))
        x = self.act2(self.dropout2(self.fc2(x)))
        x = self.fc3(x)
        return x


  

In [0]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
from rake_nltk import Rake
from nltk.corpus import stopwords

stop = set(stopwords.words('english'))

rakey = Rake(max_length=3)

def encode(sentence):
  return sentence


def get_similarity_score(sentence1, sentence2):
    # doesn't do anything about frequency of words in a sentence
    tokens1 = set(re.findall(r'[\w]+', sentence1.lower()))
    tokens2 = set(re.findall(r'[\w]+', sentence2.lower()))
    # print(sentence1)
    rakey.extract_keywords_from_sentences(sentence1.replace("-", " ").lower().split())

    keys1 = rakey.get_ranked_phrases()
    #keys1 = [x.split() for x in keys1]
    # print(keys1)
    #keys1 = [item for sublist in keys1 for item in sublist]
    # print(keys1)

    rakey.extract_keywords_from_sentences(sentence2.replace("-", " ").lower().split())
    keys2 = rakey.get_ranked_phrases()
    #keys2 = [x.split() for x in keys2]
    #keys2 = [item for sublist in keys2 for item in sublist]

    #keys = set(keys1) and set(keys2)
    tokens1 = set(keys1) - stop #- set(ref_keys)
    tokens2 = set(keys2) - stop #- set(cite_keys)
    #print(tokens1)
    #print(tokens2)
    #wefwefwef
    # tokens1 = tokens1 - stop
    # tokens2 = tokens2 - stop
  
    return len(tokens1.intersection(tokens2)) / len(tokens1.union(tokens2))

In [0]:
def pad_to_max(sentences):
  max_len = max([len(x) for x in sentences])
  sentences = [np.pad(x, (0, max_len - len(x)), mode='constant', constant_values=3) for x in sentences]
  return sentences

In [0]:
rakey = Rake(max_length=1)
rakey.extract_keywords_from_sentences("We introduce a morphological tagger which are used to tag important documents".lower().split())
keys2 = rakey.get_ranked_phrases()
print(set(keys2)-stop)

{'tag', 'introduce', 'important', 'tagger', 'used', 'documents', 'morphological'}


In [0]:
model = CiteModel()
model.cuda()

model.load_state_dict(torch.load('/content/drive/My Drive/QA/model2.pt'))
model.eval()

In [0]:
from tqdm import tqdm_notebook as tqdm
accuracy = 0
total = 0
empty_citations = 0
empty_references = 0
import pickle

with open("/content/drive/My Drive/processed-data-2019.json", 'rb') as f:
  dataset = pickle.load(f)

with open("/content/drive/My Drive/processed-data-2018.json", 'rb') as f:
  dataset2 = pickle.load(f)

dataset =  dataset2 + dataset

articles = [x[1] for x in dataset]
n = 5



pbar = tqdm(dataset)
tp = 0
fp = 0
fn = 0
avg_score = 0
all_scores = []
for data in pbar:
    ref_article = data.ref
    citing_article = data.cite
    offsets = data.offsets
    citing_sentence_ids = offsets.cite
    true_ref_sentences = offsets.ref
    true_ref_sentence_ids = offsets.ref
    if tp > 0:
      p = tp/(max(tp+fp,1))
      r = tp/(max(tp+fn,1))
      f1 = 2*p*r/(p+r)
      pbar.set_description("Processing %.3f %.3f %.3f" %(p, r, f1))
    if citing_article.sentences:
      # raku.extract_keywords_from_sentences([citing_article.sentences[x] for x in citing_article.sentences])
      # cite_keys = raku.get_ranked_phrases()[:0]
      # raku.extract_keywords_from_sentences([ref_article.sentences[x] for x in ref_article.sentences])
      # ref_keys = raku.get_ranked_phrases()[:0]

      new_ids = [c for c in citing_sentence_ids]
      for c in citing_sentence_ids:
            # If additional context is reqd
            to_add  = 0
            extra = range(max(1, c-to_add), c)
            new_ids.extend(extra)
            extra = range(c+1, min(len(citing_article.sentences), c + to_add+1))
            new_ids.extend(extra)
      citing_sentence_ids = new_ids
      complete_citing_sentence = " ".join([citing_article.sentences[c] for c in citing_sentence_ids])
      ref_vs = [(x[0],encode(x[1])) for x in ref_article.sentences.items() if abs(len(x[1].split()) - len(complete_citing_sentence.split())) <=40]
      similarity_score = {}
      for i in range(1):#c in citing_sentence_ids:
        complete_citing_sentence = ' '.join([citing_article.sentences[x] for x in citing_sentence_ids])
        complete_citing_sentence = citing_article.sentences[c]
        #complete_citing_sentence = re.sub('\((.+?)\)', '', complete_citing_sentence)
        complete_citing_sentence = encode(complete_citing_sentence)
        for ref_id, ref_sentence in ref_vs: #(ref_article.sentences.items()):
            try:
                similarity_score[ref_id] = max(similarity_score.get(ref_id, 0) , get_similarity_score(ref_sentence, complete_citing_sentence))
            except Exception as e:
                # print(e)
                pass
      if similarity_score:
          sorted_similarity_score = sorted(similarity_score.items(), key=lambda item: -item[1])
          top_n = [s for s in sorted_similarity_score]
          top_n = {x[0]:x[1] for x in top_n[:5]}
          # if len(top_n) > 1:
          #   top_n = rerank2(top_n, ref_article, ' '.join([citing_article.sentences[x] for x in citing_sentence_ids]))

          fp += len(top_n)
          for x in true_ref_sentence_ids:
            if x in top_n:
              avg_score = (tp*avg_score + top_n[x])/(max(tp,1))
              all_scores.append(top_n[x])
              fp -= 1
              tp += 1
            else:
              #print(ref_article.sentences[x])
              #print([citing_article.sentences[x] for x in citing_sentence_ids])
              fn += 1
print(tp, fp, fn)
      


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


In [0]:
!pip install gensim
import gensim.downloader as api
model = api.load("glove-wiki-gigaword-300")


[==================================================] 100.0% 376.1/376.1MB downloaded


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
s1 = 'This room is dirty'
s2 = 'dirty and disgusting room' #corrected variable name
print(model.wv.n_similarity(s1.lower().split(), s2.lower().split()))


0.80902576


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [0]:
import torch.nn.functional as F
def rerank(top_n, ref_article, cite):
  with torch.no_grad():
    scores = {}
    refs = []
    cites = []
    ref_article, ref_sections = ref_article.sentences, ref_article.sections
    try:
      ref_title = ref_article[0]
    except:
      ref_title = "No title"
    id_map = {}
    i = 0
    for x in top_n:
      x1 = (" ".join([ref_article[x]]),
                            cite, 1, ref_title.strip(),
                            ref_sections[x].strip() if any(c.isalpha() for c in ref_sections[x].strip()) else "##other##")
      ref = (" " + x1[3]+x1[4]+x1[0])[:512]
      cite = " " + x1[1][:512]
      refs.append(ref)
      cites.append(cite)
      id_map[x] = i
      i += 1
    results = model.forward(refs, cites)
    results = F.softmax(results)
    results = results[:,1].cpu().detach().numpy().tolist()
    top_n_v = top_n.items()
    # print(top_n_v)
    top_n_v = sorted(top_n_v, key = lambda x : results[id_map[x[0]]], reverse=True)
    top_n_v = [x for x in top_n_v if results[id_map[x[0]]] > 0.27]
    top_n_v = top_n_v[:3]
    top_n = {x:y for x,y in top_n_v}
    return top_n


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
all_scores


In [0]:

import torch
from torch.nn.functional import softmax
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from transformers import BertForNextSentencePrediction, BertTokenizer, AdamW
import json
import multiprocessing
import numpy as np

model = BertForNextSentencePrediction.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

model.to(device)

model.load_state_dict(torch.load("/content/drive/My Drive/model.npy"))

<All keys matched successfully>

In [0]:
import torch.nn.functional as F
def rerank2(top_n, ref_article, cite):
  with torch.no_grad():
    scores = {}
    refs = []
    cites = []
    ref_article, ref_sections = ref_article.sentences, ref_article.sections
    try:
      ref_title = ref_article[0]
    except:
      ref_title = "No title"
    id_map = {}
    i = 0
    for x in top_n:
      x1 = (" ".join([ref_article[x]]),
                            cite, 1, ref_title.strip(),
                            ref_sections[x].strip() if any(c.isalpha() for c in ref_sections[x].strip()) else "##other##")
      ref = (" " + x1[3]+x1[4]+x1[0])[:512]
      cite = " " + x1[1][:512]
      refs.append(ref)
      cites.append(cite)
      id_map[x] = i
      i += 1
    
    pairofstrings = list(zip([x.lower() for x in refs], [x.lower() for x in cites]))
    encoded_batch = tokenizer.batch_encode_plus(pairofstrings, add_special_tokens=True, return_tensors='pt',max_length=512,
                                                            return_special_tokens_masks=True, pad_to_max_length=True)
    attention_mask = (encoded_batch['attention_mask'] - encoded_batch['special_tokens_mask']).to(device)
    input_ids, token_type_ids = encoded_batch['input_ids'].to(device), encoded_batch['token_type_ids'].to(device)
    results = model(input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)[0] 
    results = F.softmax(results)
    results = results[:,1].cpu().detach().numpy().tolist()
    top_n_v = top_n.items()
    # print(top_n_v)
    top_n_v = sorted(top_n_v, key = lambda x : results[id_map[x[0]]], reverse=True)
    top_n_v = [x for x in top_n_v if results[id_map[x[0]]] > 0.1 ]
    top_n_v = top_n_v[:3]
    top_n = {x:y for x,y in top_n_v}
    return top_n

In [0]:
sorted(all_scores)[len(all_scores)//2]

0.7334244

In [0]:
dataset[0]

Datum(ref=Article(xml=<xml.etree.ElementTree.ElementTree object at 0x7ff71e1e0d68>, sentences={0: 'OVERVIEW OF RESULTS OF THE MUC-6 EVALUATION', 1: 'Test abstract', 2: 'The latest in a series of natural language processing system evaluations was concluded in October 1995 and was the topic of the Sixth Message Understanding Conference (MUC6) in November.', 3: 'Participants were invited to enter their systems in as many as four different task-oriented evaluations.', 4: 'The Named Entity and Coreference tasks entailed Standard Generalized Markup Language (SGML) annotation of texts and were being conducted for the first time.', 5: 'The other two tasks, Template Element and Scenario Template, were information extraction tasks that followed on from the MUC evaluations conducted in previous years.', 6: 'The evolution and design of the MUC6 evaluation are discussed in the paper by Grishman and Sundheim in this volume.', 7: 'All except the Scenario Template task are defined independently of any

In [0]:
xy = {}
for data in dataset:
  if not data.cite.sentences or not data.ref.sentences:
        continue
  y = data.facet
  ref_sents = [''.join([data.ref.sentences[x] for x in data.offsets.ref])]
  cite_sents = [''.join([data.cite.sentences[x] for x in data.offsets.cite])]
  for sent in ref_sents + cite_sents:
      xy[sent] = y


In [0]:
len(xy)

1123

In [0]:
f = {}
for x in xy:
  y = xy[x]
  y_ = "_".join(sorted([x.lower().replace("_", "").replace("s", "").replace(" ", "") for x in y]))
  #print(y)
  if "aim" in "".join(y).lower():
    print(x)
    print("----")
  f[y_] = f.get(y_, 0) + 1

In [0]:
f


{'aimcitation': 59,
 'aimcitation_hypotheicitation': 5,
 'aimcitation_methodcitation': 25,
 'aimcitation_reultcitation': 4,
 'hypotheicitation': 19,
 'hypotheicitation_methodcitation': 6,
 'implicationcitation': 80,
 'implicationcitation_methodcitation': 14,
 'implicationcitation_reultcitation': 4,
 'methodcitation': 754,
 'methodcitation_reultcitation': 35,
 'reultcitation': 118}

In [0]:
# -*- coding: utf-8 -*-
"""Untitled2.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1OVxFNbLNcEcJnoNn80hhp2SC_Rk_QzD8
"""

from collections import namedtuple

import requests

Datum = namedtuple('Datum', 'ref cite offsets author is_test facet year')
Offsets = namedtuple('Offsets', 'marker cite ref')
Article = namedtuple('Article', 'xml sentences sections')

# Currently only works for 2016 - 2018

import os
import xml.etree.cElementTree as ET
import logging
import codecs
from tqdm import tqdm as tqdm
import re

DATA_ROOT = '/content/drive/My Drive/data'

sep = os.path.sep
ET.XMLParser(encoding="utf-8")

paper_load_fail = 0
annotation_load_fail = 0

l = logging.getLogger('load_parse')


# Gets names of folders where data is present. ex : "./data/Training-Set-2016/C90-2039_TRAIN"
def get_folders(data_root):
    folders = []
    global folder_skips
    for year_folder in filter(lambda x: os.path.isdir(data_root + sep + x), os.listdir(data_root)):
        year_folder = data_root + sep + year_folder
        for data_folder in os.listdir(year_folder):
            data_folder = year_folder + sep + data_folder
            if not os.path.isdir(
                    data_folder) or '2019' in data_folder or 'Test-Set-2018' in data_folder or '2018' not in data_folder:
                continue
                # readme
            folders.append(data_folder)
    return folders


def load_article(filename):
    global paper_load_fail
    if not os.path.splitext(filename)[1] == '.xml':
        l.info("Skipping Non xml : " + filename)
        return
    l.info("parsing :" + filename)
    try:
        # Ignoring non UTF characters for now
        with codecs.open(filename, mode='r', encoding='utf-8', errors='ignore') as target_file:

            xml = ET.parse(target_file)
            parent_map = {c: p for p in xml.getroot().iter() for c in p}
            sentence_elements = list(xml.getroot().iter('S'))
            sentence_elements = [(x.text,
                                  parent_map[x].attrib['title'] if len(parent_map[x].attrib) > 0 else parent_map[x].tag,
                                  int(x.attrib['sid']) if 'sid' in x.attrib else 99)
                                 for x in
                                 sentence_elements]
            # TODO: Check if this is too memory inefficient. Should mostly be okay
            sentence_map = {x[2]: x[0] for i, x in enumerate(sentence_elements)}
            section_map = {x[2]: x[1] for i, x in enumerate(sentence_elements)}
            article = Article(xml, sentence_map, section_map)
            return article
    except Exception as e:
        l.error("Error with : " + filename + " with ex : " + str(e))
        paper_load_fail += 1
        return Article(ET.fromstring("<xml></xml>"), {}, {})


def newest_file(path):
    files = os.listdir(path)
    paths = [os.path.join(path, basename) for basename in files]
    return max(paths, key=os.path.getctime)

author_memo = {}

def load_folder_data(annotation_file, articles, is_test=False):
    global annotation_load_fail
    data = []
    global author_memo

    year_matches = re.findall(".*Set[-]([0-9]{4}).*", annotation_file)
    year = int(year_matches[0]) if len(year_matches) > 0 else 2016  # Dev set
    with open(annotation_file, 'r') as f:
        for line in f:
            if len(line.strip()) == 0:
                continue
            try:
                parts = line.split(" | ")
                parts = [part.strip() for part in parts]
                ref_article_name = parts[1].split(":")[1].strip().upper().replace(".XML", "").replace(".TXT", "")
                if ref_article_name == "P07-1040":
                    print("gg")
                ref_article = articles[parts[1].split(":")[1].strip().upper().replace(".XML", "").replace(".TXT", "")]
                cite_article = articles["-".join(
                    parts[2].split(":")[1].strip().upper().replace("_", "-").replace(".XML", "").replace(".TXT",
                                                                                                         "").split("-")[
                    :2])]
                marker_offset = [int(x) for x in (
                    parts[5].split(":")[1].replace("'", "").replace("[", "").replace("]", "").strip()).replace(",",
                                                                                                               " ").split()]
                citation_offsets = [int(x) for x in (
                    parts[5].split(":")[1].replace("'", "").replace("[", "").replace("]", "").strip()).replace(",",
                                                                                                               " ").split()]
                ref_offsets = [int(x) for x in (
                    parts[7].split(":")[1].replace("'", "").replace("[", "").replace("]", "").strip()).replace(",",
                                                                                                               " ").split()]
                facet = parts[9].split(":")[1].strip()
                facet = [facet] if '[' not in facet else eval(facet)
                author = '' if len(parts) < 10 or ":" not in parts[10] else parts[10].split(":")[1].replace("|",
                                                                                                            "").strip()
                if ref_article_name not in author_memo:
                    url = "https://www.aclweb.org/anthology/{}.bib".format(ref_article_name)
                    info = requests.get(url=url).text
                    author_info = info.split('{')[1].split(',')[0].split('-')
                    author = get_formatted_author_info(author_info)
                    author_memo[ref_article_name] = author_info
                else:
                    print("Hit")
                    author_info = author_memo[ref_article_name]
                    author = get_formatted_author_info(author_info)
                year = author_info[-2]
                ref_article, cite_article = get_clean_cite_and_ref(ref_article, cite_article, ref_offsets, citation_offsets,
                                                         author, year)

                d = Datum(ref_article, cite_article, Offsets(marker_offset, citation_offsets, ref_offsets), author,
                          is_test, facet, year)
                data.append(d)
            except Exception as e:
                l.error(e)
                annotation_load_fail += 1
    return data


def load_folder(folder_root):
    annotation_folder = folder_root + sep + "annotation"
    citance_dir = folder_root + sep + "Citance_XML"
    ref_name = folder_root.split(sep)[-1].replace("_TRAIN", "")
    ref_dir = folder_root + sep + "Reference_XML" + sep + ref_name + ".xml"
    articles = {
        '-'.join(os.path.splitext(x)[0].upper().replace("_", "-").split("-")[:2]): load_article(citance_dir + sep + x)
        for x in os.listdir(citance_dir)}

    assert len(articles) == len(os.listdir(citance_dir))

    ref_article = load_article(ref_dir)
    articles[ref_name.upper()] = ref_article
    annotation_file = newest_file(annotation_folder)
    folder_data = load_folder_data(annotation_file, articles)
    return folder_data

import copy

replace_count = 0

def get_clean_cite_and_ref(ref_article, cite_article, ref_offsets, citation_offsets, author, year):
    global replace_count
    ref = copy.copy(ref_article)
    cite = copy.copy(cite_article)
    cite_sentence = " ".join([cite.sentences[c] for c in citation_offsets])
    cite_sentence = re.sub(r"\D(\d{4})\D", '', cite_sentence)  # regex for removing years
    cite_sentence = re.sub(r"\[[0-9]{1,3}\]", '', cite_sentence)  # regex for removing citation numbers
    translation = {ord(')'): None, ord('('): None, ord('.'): None,  ord(','): None, ord('!'): 'l'}
    cite_sentence = cite_sentence.translate(translation)
    author_info = author.split(" ")
    author_2 = None
    if author_info[-1] in ["et.al.", "etal", "Etal"]:
        author_1 = " ".join(author_info[:-1]) + " et al"
        if len(author_info) >= 2:
            author_2 = author_info[0] + " and " + author_info[1]
    elif len(author_info) >= 2 and author_info[-2] not in ["And", "and"]:
        author_1 = author_info[0] + " & " + author_info[1]
        author_2 = author_info[0] + " and " + author_info[1]
    else:
        author_1 = author_info[-1]

    citing_paper_text = author_1
    old_cite_sentence = cite_sentence
    cite_sentence = cite_sentence.replace(citing_paper_text, "##CITATION##")
    if cite_sentence != old_cite_sentence:
        replace_count += 1
    elif author_2:
        citing_paper_text = author_2
        cite_sentence = cite_sentence.replace(citing_paper_text, "##CITATION##")
        if cite_sentence != old_cite_sentence:
            replace_count += 1
        elif len(author_info) >= 2 and author_info[-2] not in ["And", "and"]:
            author_info.sort()
            author_1 = author_info[0] + " & " + author_info[1]
            author_2 = author_info[0] + " and " + author_info[1]
            citing_paper_text = author_1
            old_cite_sentence = cite_sentence
            cite_sentence = cite_sentence.replace(citing_paper_text, "##CITATION##")
            if cite_sentence != old_cite_sentence:
                replace_count += 1
            elif author_2:
                citing_paper_text = author_2
                cite_sentence = cite_sentence.replace(citing_paper_text, "##CITATION##")
                if cite_sentence != old_cite_sentence:
                    replace_count += 1
                else:
                    print("cite_sentence: {}\n old_cite_sentence: {}\n author: {}".format(cite_sentence,
                                                                                          old_cite_sentence, author))
            else:
                print("cite_sentence: {}\n old_cite_sentence: {}\n author: {}".format(cite_sentence,
                                                                                      old_cite_sentence, author))
        else:
            print("cite_sentence: {}\n old_cite_sentence: {}\n author: {}".format(cite_sentence, old_cite_sentence, author))
    else:
        print("cite_sentence: {}\n old_cite_sentence: {}\n author: {}".format(cite_sentence, old_cite_sentence, author))

    cite.sentences[citation_offsets[0]] = cite_sentence
    if len(citation_offsets) > 1:
        for offset in citation_offsets[1:]:
            cite.sentences[offset] = ""
    print("replace_count: ", replace_count)
    return ref, cite


def get_formatted_author_info(author_info):
    author_details = " ".join(author_info)
    author_info = re.sub(r"[0-9]{4}", '_', author_details).split('_')[0].split()
    author = " ".join([x.capitalize() if x is not "and" else x for x in author_info])
    return author





def load_all(root):
    print("Loading all folders, skipping 2019 and test set 2018 for now")
    folders = get_folders(root)
    print("Going to load :", len(folders))
    dataset = [load_folder(folder) for folder in tqdm(folders)]
    dataset = [data for datalist in dataset for data in datalist]
    print()
    print("Paper load fails due to xml errors : ", paper_load_fail)
    print("Annotation load fails due to random reasons", annotation_load_fail)
    print("Overall loaded ", len(dataset), " datapoints")
    return dataset


if __name__ == "__main__":
    logging.basicConfig(filename='example.log', level=logging.ERROR, filemode='w')
    print("Logging to example.log")
dataset = load_all(DATA_ROOT)

dataset[0]

from sentence_transformers import SentenceTransformer
from scipy.spatial.distance import cosine as cosine_similarity

embedder = SentenceTransformer('bert-base-nli-mean-tokens')


def encode(text):
    text = text.lower()
    v = embedder.encode([text])[0]
    return v


def get_similarity_score(sentence1, sentence2):
    if type(sentence1) == type(''):
        sentence1 = encode(sentence1)
    if type(sentence2) == type(''):
        sentence2 = encode(sentence2)
    return 1 - cosine_similarity(sentence1, sentence2)

#
# v1 = encode("Test sentence")
#
# v2 = encode("Test sentence")
#
# v3 = encode("Random sentence")
#
# print(get_similarity_score(v1, v3))
# print(get_similarity_score(v1, v2))

from rake_nltk import Rake

rakey = Rake(max_length=1)


# def encode(sentence):
#     return sentence
#
#
# def get_similarity_score(sentence1, sentence2):
#     # doesn't do anything about frequency of words in a sentence
#     tokens1 = set(re.findall(r'[\w]+', sentence1.lower()))
#     tokens2 = set(re.findall(r'[\w]+', sentence2.lower()))
#     # print(sentence1)
#     rakey.extract_keywords_from_sentences(sentence1.replace("-", " ").lower().split())
#
#     keys1 = rakey.get_ranked_phrases()
#     # print(keys1)
#
#     rakey.extract_keywords_from_sentences(sentence2.replace("-", " ").lower().split())
#     keys2 = rakey.get_ranked_phrases()
#
#     # keys = set(keys1) and set(keys2)
#     tokens1 = set(keys1)
#     tokens2 = set(keys2)
#     # print(tokens1)
#     # print(tokens2)
#     # wefwefwef
#
#     return len(tokens1.intersection(tokens2)) / len(tokens1.union(tokens2))
#

import re

# 106 115 123

# def encode(sentence):
#     return sentence
#
#
# def get_similarity_score(sentence1, sentence2):
#     sentence1 = re.findall(r'[\w]+', sentence1.lower())
#     sentence2 = re.findall(r'[\w]+', sentence2.lower())
#     len_sentence1 = len(sentence1)
#     len_sentence2 = len(sentence2)
#
#     if not len_sentence1 or not len_sentence2:
#         return 0.0
#
#     if sentence1 == sentence2:
#         return 1.0
#
#     if len_sentence1 == 1 or len_sentence2 == 1:
#         return 0.0
#
#     sentence1_bigrams = [sentence1[i:i + 2] for i in range(len_sentence1 - 1)]
#     sentence2_bigrams = [sentence2[i:i + 2] for i in range(len_sentence2 - 1)]
#
#     sentence1_bigrams.sort()
#     sentence2_bigrams.sort()
#
#     len_sentence1 = len(sentence1_bigrams)
#     len_sentence2 = len(sentence2_bigrams)
#
#     matches = i = j = 0
#     while i < len_sentence1 and j < len_sentence2:
#         if sentence1_bigrams[i] == sentence2_bigrams[j]:
#             matches += 2
#             i += 1
#             j += 1
#         elif sentence1_bigrams[i] < sentence2_bigrams[j]:
#             i += 1
#         else:
#             j += 1
#
#     return (2 * matches) / (len_sentence1 + len_sentence2)


accuracy = 0
total = 0
empty_citations = 0
empty_references = 0

articles = [x[1] for x in dataset]
n = 5

pbar = tqdm(dataset)
tp = 0
fp = 0
fn = 0
avg_score = 0
all_scores = []
for data in pbar:
    ref_article = data.ref
    citing_article = data.cite
    offsets = data.offsets
    citing_sentence_ids = offsets.cite
    true_ref_sentences = offsets.ref
    true_ref_sentence_ids = offsets.ref
    if tp > 0:
        p = tp / (max(tp + fp, 1))
        r = tp / (max(tp + fn, 1))
        f1 = 2 * p * r / (p + r)
        pbar.set_description("Processing %.3f %.3f %.3f" % (p, r, f1))

    if citing_article.sentences:
        new_ids = [c for c in citing_sentence_ids]
        for c in citing_sentence_ids:
            # If additional context is reqd
            to_add = 0
            extra = range(max(1, c - to_add), c)
            new_ids.extend(extra)
            extra = range(c + 1, min(len(citing_article.sentences), c + to_add + 1))
            new_ids.extend(extra)
        citing_sentence_ids = new_ids
        # complete_citing_sentence = " ".join([citing_article.sentences[c] for c in citing_sentence_ids])
        ref_vs = [(x[0], encode(x[1])) for x in ref_article.sentences.items()]
        similarity_score = {}
        for c in citing_sentence_ids:
            complete_citing_sentence = citing_article.sentences[c]
            complete_citing_sentence = encode(complete_citing_sentence)
            for ref_id, ref_sentence in ref_vs:  # (ref_article.sentences.items()):
                try:
                    similarity_score[ref_id] = max(similarity_score.get(ref_id, 0),
                                                   get_similarity_score(ref_sentence, complete_citing_sentence))
                except Exception as e:
                    # print(e)
                    pass
        if similarity_score:
            sorted_similarity_score = sorted(similarity_score.items(), key=lambda item: -item[1])
            top_n = [s for s in sorted_similarity_score]
            top_n = {x[0]: x[1] for x in top_n[:3]}
            fp += len(top_n)
            for x in true_ref_sentence_ids:
                if x in top_n:
                    avg_score = (tp * avg_score + top_n[x]) / (max(tp, 1))
                    all_scores.append(top_n[x])
                    fp -= 1
                    tp += 1
                else:
                    # print(ref_article.sentences[x])
                    # print([citing_article.sentences[x] for x in citing_sentence_ids])
                    fn += 1
print(tp, fp, fn)

xy = {}
for data in dataset:
    if not data.cite.sentences or not data.ref.sentences:
        continue
    y = data.facet
    ref_sents = [''.join([data.ref.sentences[x] for x in data.offsets.ref])]
    cite_sents = [''.join([data.cite.sentences[x] for x in data.offsets.cite])]
    for sent in ref_sents + cite_sents:
        xy[sent] = y

len(xy)

f = {}
for x in xy:
    y = xy[x]
    y_ = "_".join(sorted([x.lower().replace("_", "").replace("s", "").replace(" ", "") for x in y]))
    # print(y)
    if "aim" in "".join(y).lower():
        print(x)
        print("----")
    f[y_] = f.get(y_, 0) + 1





Logging to example.log
Loading all folders, skipping 2019 and test set 2018 for now


  0%|          | 0/40 [00:00<?, ?it/s]

Going to load : 40


  2%|▎         | 1/40 [00:06<04:22,  6.74s/it]

replace_count:  1
Hit
replace_count:  2
Hit
replace_count:  3
Hit
cite_sentence: Algorit hmThis algorithm was first implemented for the MUC 6 FASTUS system Appelt et al  and prod uced one of the top scores a recall of 59% and precision of 72% in the MUC6 Coreference Task which evaluated systems' ability to recog nize coreference among noun phrases Sund heim
 old_cite_sentence: Algorit hmThis algorithm was first implemented for the MUC 6 FASTUS system Appelt et al  and prod uced one of the top scores a recall of 59% and precision of 72% in the MUC6 Coreference Task which evaluated systems' ability to recog nize coreference among noun phrases Sund heim
 author: Sundheim
replace_count:  3
Hit
replace_count:  4
Hit
replace_count:  5
Hit
replace_count:  6
Hit
replace_count:  7
Hit
replace_count:  8
Hit
cite_sentence: The organizers of MUC 6 did not attempt to compare the difficulty of the MUC 6 task to the previous MUC tasks saying that the problem of coming up with a reasonable objective w

...

 25%|██▌       | 10/40 [02:27<08:23, 16.78s/it]

cite_sentence: And nally TAGPAIR uses classication pair weights based on the probability of a classication for some predicted classication pair van Halteren et al
 old_cite_sentence: And nally TAGPAIR uses classication pair weights based on the probability of a classication for some predicted classication pair van Halteren et al
 author: Van Halteren Etal
replace_count:  145
Hit
replace_count:  146
Hit
replace_count:  147
Hit
cite_sentence: We consider three voting strategies suggested by van Halteren et al : equal vote where each classifier's vote is weighted equally overall accuracy where the weight depends on the overall accuracy of a classifier and pair'wise voting
 old_cite_sentence: We consider three voting strategies suggested by van Halteren et al : equal vote where each classifier's vote is weighted equally overall accuracy where the weight depends on the overall accuracy of a classifier and pair'wise voting
 author: Van Halteren Etal
replace_count:  147
Hit
cite_sentence: Hal

 28%|██▊       | 11/40 [02:34<06:42, 13.89s/it]

replace_count:  152
Hit
cite_sentence: Gerdemann and G6tz's Troll system see [G6Tz [GFRDEIVIANN AND KING and [GERDEMANN FC] employs an efficient refinement of RES to test the satisfiability of feature structures In fact Troll represents each feature structure as a disjunction of the resolvants of the feature structure Loosely speaking the resolvants of a feature structure have the same underlying finite state automaton as the feature structure and differ only in their output fllnction Troll exploits this property to represent each feature structure as a finite state automaton and a set of output flmctions The '1oll unifier is closed on these representations Thus though RES is computationally expensive Troll uses RES only during compilation never during run time
 old_cite_sentence: Gerdemann and G6tz's Troll system see [G6Tz [GFRDEIVIANN AND KING and [GERDEMANN FC] employs an efficient refinement of RES to test the satisfiability of feature structures In fact Troll represents each featu

KeyboardInterrupt: ignored

In [0]:
# latest tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.metrics.pairwise import polynomial_kernel
import pickle
import os

global tf_vectorizer


def build_model(cite_paper, ref_paper, n=1):
    tf = TfidfVectorizer(analyzer='word', ngram_range=(1, n), min_df=0, stop_words='english')
    sentences = list(cite_paper.sentences.values())
    sentences.extend(list(ref_paper.sentences.values()))
    tf_vectorizer = tf.fit(sentences)
    pickle.dump(tf_vectorizer, open("tf-idf_cache/tf_cite_{}_ref_{}_n_{}.pickle".format(cite_paper.id, ref_paper.id, n), 'wb'))
    return tf_vectorizer

def encode(sentence):
  return sentence

def get_similarity_score(cite_paper, ref_paper, sentence1, sentence2, n=1, kernel="none"):
    if os.path.exists("tf-idf_cache/tf_cite_{}_ref_{}_n_{}.pickle".format(cite_paper.id, ref_paper.id, n)):
        tf_vectorizer = pickle.load(open("tf-idf_cache/tf_cite_{}_ref_{}_n_{}.pickle".format(cite_paper.id, ref_paper.id, n), 'rb'))
    else:
        tf_vectorizer = build_model(cite_paper, ref_paper, n)

    tfidf_1 = tf_vectorizer.transform([sentence1])
    tfidf_2 = tf_vectorizer.transform([sentence2])
    if kernel == "none":
        return cosine_similarity(tfidf_1, tfidf_2).item()
    elif kernel == "linear":
        return linear_kernel(tfidf_1, tfidf_2).item()
    elif kernel == "poly_2":
        return polynomial_kernel(tfidf_1, tfidf_2, 2).item()
    elif kernel == "poly_3":
        return linear_kernel(tfidf_1, tfidf_2, 3).item()
    elif kernel == "rbf":
        return rbf_kernel(tfidf_1, tfidf_2).item()

In [0]:
# latest bm25
!pip install rank_bm25

from rank_bm25 import BM25Okapi
import pickle
import os

global bm25

def build_model(ref_paper):
    sentences = ref_paper.sentences.items()
    tokenized_sentences = [doc.split(" ") for doc in sentences]
    bm25 = BM25Okapi(tokenized_sentences)
    pickle.dump(bm25, open("bm_25/bm_25_ref_{}.pickle".format(ref_paper.id), 'wb'))
    return bm25

def get_similarity_score(ref_paper, ref_sentence_id, sentence2):
    if os.path.exists("bm_25/bm_25_ref_{}.pickle".format(ref_paper.id)):
        bm25 = pickle.load(open("bm_25/bm_25_ref_{}.pickle".format(ref_paper.id), 'rb'))
    else:
        bm25 = build_model(ref_paper)

    tokenized_query = sentence2.split(" ")
    scores = bm25.get_scores(tokenized_query)
    score = scores[ref_sentence_id]
    return score


  Created wheel for rank-bm25: filename=rank_bm25-0.2-cp36-none-any.whl size=4162 sha256=d4eb3e115d25725d7a486ff3242ad6a721ceb8796674bdd488e4f265922c9ff4
  Stored in directory: /root/.cache/pip/wheels/6f/0c/1f/78945dd6a5478bbcdb50d73ac96ae5af2ffcdfcd374fd9b1bf
Successfully built rank-bm25


In [0]:
import nltk
import re


def get_similarity_score(sentence1, sentence2, n=1, type=1):
    # doesn't do anything about frequency of words in a sentence
    if type == 1:

        if n == 1:
            tokens1 = set(re.findall(r'[\w]+', sentence1.lower()))
            tokens2 = set(re.findall(r'[\w]+', sentence2.lower()))
            return len(tokens1.intersection(tokens2)) / len(tokens1.union(tokens2))

        elif n == 2:
            bigrams1 = set(nltk.bigrams([word for word in sentence1.lower().split()]))
            bigrams2 = set(nltk.bigrams([word for word in sentence2.lower().split()]))
            return len(bigrams1.intersection(bigrams2)) / len(bigrams1.union(bigrams2))

        elif n == 3:
            trigrams1 = set(nltk.trigrams([word for word in sentence1.lower().split()]))
            trigrams2 = set(nltk.trigrams([word for word in sentence2.lower().split()]))
            return len(trigrams1.intersection(trigrams2)) / len(trigrams1.union(trigrams2))

    elif type == 2:

        if n == 1:
            tokens1 = set(re.findall(r'[\w]+', sentence1.lower()))
            tokens2 = set(re.findall(r'[\w]+', sentence2.lower()))
            return len(tokens1.intersection(tokens2)) / len(tokens1)

        elif n == 2:
            bigrams1 = set(nltk.bigrams([word for word in sentence1.lower().split()]))
            bigrams2 = set(nltk.bigrams([word for word in sentence2.lower().split()]))
            return len(bigrams1.intersection(bigrams2)) / len(bigrams1)

        elif n == 3:
            trigrams1 = set(nltk.trigrams([word for word in sentence1.lower().split()]))
            trigrams2 = set(nltk.trigrams([word for word in sentence2.lower().split()]))
            return len(trigrams1.intersection(trigrams2)) / len(trigrams1)

    elif type == 3:

        if n == 1:
            tokens1 = set(re.findall(r'[\w]+', sentence1.lower()))
            tokens2 = set(re.findall(r'[\w]+', sentence2.lower()))
            return len(tokens1.intersection(tokens2)) / len(tokens2)

        elif n == 2:
            bigrams1 = set(nltk.bigrams([word for word in sentence1.lower().split()]))
            bigrams2 = set(nltk.bigrams([word for word in sentence2.lower().split()]))
            return len(bigrams1.intersection(bigrams2)) / len(bigrams2)

        elif n == 3:
            trigrams1 = set(nltk.trigrams([word for word in sentence1.lower().split()]))
            trigrams2 = set(nltk.trigrams([word for word in sentence2.lower().split()]))
            return len(trigrams1.intersection(trigrams2)) / len(trigrams2)
